# 解析PDF

为了保证数据干净，需要把处理好的txt中前9行和 倒数前四行删掉


In [4]:
%pip install PyMuPDF PyPDF2 tiktoken accelerate>=0.26 baiduspider requests>=2.24.0 bs4>=0.0.1

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from Chunk.datachunk import TextChunker
from Embedding.model import ChineseBERTEmbedding,BGEReranker
from VectorDatabase.database import Vectordatabase
from transformers import BertTokenizer
import json
import fitz  # PyMuPDF
import re  # 引入正则表达式模块，用于清理特殊字符
from transformers import BertForPreTraining, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

import time ,jwt,requests
from pprint import pprint


e:\miniconda\envs\RAG\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
e:\miniconda\envs\RAG\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [14]:
def pdf_to_txt(pdf_path, txt_path):
    try:
        # 打开 PDF 文件
        pdf_document = fitz.open(pdf_path)
        text_content = ""

        # 遍历每一页提取文本 : 从第12页开始，后面4页是广告
        for page_num in range(11,len(pdf_document) - 4):
            page = pdf_document[page_num]
            text_content += page.get_text() + "\n"  # 获取当前页的文字内容

        # 将提取的文本保存到 TXT 文件
        with open(txt_path, "w", encoding="utf-8") as txt_file:
            txt_file.write(text_content)

        print(f"成功将 PDF 内容保存为 TXT 文件：{txt_path}")
    except Exception as e:
        print(f"发生错误：{e}")

# 文件路径
pdf_file = "Book/齐民要术-石声汉译注.pdf"  # 输入 PDF 文件路径
txt_file = "Book/齐民要术-石声汉译注.txt"  # 输出 TXT 文件路径

# 执行转换
pdf_to_txt(pdf_file, txt_file)

成功将 PDF 内容保存为 TXT 文件：Book/齐民要术-石声汉译注.txt


In [15]:
def clean_and_combine_txt(input_path, output_path):
    try:
        with open(input_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # 删除空行并去掉每行的首尾空格
        cleaned_lines = [line.strip() for line in lines if line.strip()]

        # 清除特殊字符（保留中文、英文、数字和常见符号）
        # def remove_special_chars(text):
        #     # 保留中英文、数字、常用标点（句号、逗号等）
        #     return re.sub(r"[^\w\s\u4e00-\u9fa5.,，。！？:：；;（）()《》“”‘’\"'-]", "", text)

        # 清除特殊字符（保留中文、英文和常见标点符号，不保留数字）
        # def remove_special_chars(text):
        #     # 正则表达式：保留中英文、空格和常见标点符号，移除数字
        #     return re.sub(r"[^\w\s\u4e00-\u9fa5.,，。！？:：；;（）()《》“”‘’\"'-]|[0-9]", "", text)

        # 清除特殊字符（保留中文和常见标点符号，不保留数字和英文字母）
        def remove_special_chars(text):
            # 正则表达式：保留中文字符、空格和常见标点符号，移除英文字母和数字
            return re.sub(r"[^\u4e00-\u9fa5.,，。！？:：；;（）()《》“”‘’\"'-]", "", text)
        
        cleaned_lines = [remove_special_chars(line) for line in cleaned_lines]
        # 拼接所有行，按段落合并（用空格连接）
        combined_text = " ".join(cleaned_lines)

        # 将处理后的文本写入新文件
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(combined_text)

        print(f"成功处理文本并保存至：{output_path}")
    except Exception as e:
        print(f"发生错误：{e}")

# 文件路径
input_txt = "Book/齐民要术-石声汉译注.txt"  # 输入 TXT 文件路径
output_txt = "Book/齐民要术.txt"  # 输出处理后 TXT 文件路径

# 执行清理和拼接
clean_and_combine_txt(input_txt, output_txt)

成功处理文本并保存至：Book/齐民要术.txt


In [13]:
def analyze_text(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

        # 统计总字数（去掉空格后的字符总数）
        total_characters = len(content.replace(" ", "").replace("\n", ""))

        # 统计总行数
        lines = content.splitlines()
        total_lines = len(lines)

        # 统计段落数（按空行分割段落）
        paragraphs = [para.strip() for para in content.split("\n\n") if para.strip()]
        total_paragraphs = len(paragraphs)

        # 统计单词数（按空格分割的词汇总数）
        words = content.split()
        total_words = len(words)

        # 打印统计结果
        print(f"总字数（去空格）：{total_characters}")
        print(f"总行数：{total_lines}")
        print(f"总段落数：{total_paragraphs}")
        print(f"总单词数：{total_words}")

        # 返回统计结果（如果需要进一步使用）
        return {
            "total_characters": total_characters,
            "total_lines": total_lines,
            "total_paragraphs": total_paragraphs,
            "total_words": total_words,
        }
    except Exception as e:
        print(f"发生错误：{e}")

# 文件路径
txt_file = "Book/齐民要术.txt"  # 输入 TXT 文件路径

# 执行统计分析
stats = analyze_text(txt_file)

发生错误：UTF-16 stream does not start with BOM


# 补充材料：关于齐民要术 的百度补充

In [22]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

def search_baidu(keyword, max_results=10):
    """在百度搜索指定关键词并返回结果标题和链接"""
    url = f"https://www.baidu.com/s?wd={keyword}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # 提取搜索结果标题和链接
    results = []
    for h3 in soup.find_all("h3")[:max_results]:  # 限制最大结果数量
        title = h3.get_text(strip=True)
        link = h3.a["href"]
        results.append({"title": title, "link": link})
    
    return results

def fetch_page_content(url):
    """获取指定链接的页面内容，尝试提取<p>、<span>和<div>标签中的文字"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = response.apparent_encoding  # 自动检测页面编码
        soup = BeautifulSoup(response.text, "html.parser")

        # 提取正文内容（按优先级提取 p、span、div 标签的文字内容）
        paragraphs = soup.find_all(["p", "span", "div"])
        content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
        return content
    except Exception as e:
        print(f"无法爬取 {url}: {e}")
        return ""

def save_to_txt(file_path, content):
    """将内容保存到文本文件"""
    with open(file_path, "a", encoding="utf-8") as file:
        file.write(content + "\n\n")

keyword = "齐民要术"
output_file = "Book/齐民要术_补充材料.txt"

# 清空旧的知识库文件
with open(output_file, "w", encoding="utf-8") as file:
    file.write("")

# 搜索百度
print(f"正在搜索关键词：{keyword}")
results = search_baidu(keyword, max_results=10)
pprint(results)

# 爬取每个结果链接的内容
for i, result in enumerate(results):
    print(f"正在爬取第 {i + 1} 个链接：{result['title']}")
    content = fetch_page_content(result["link"])
    if content:
        save_to_txt(output_file, f"标题: {result['title']}\n链接: {result['link']}\n\n{content}")
    else:
        print(f"跳过：无法获取内容 {result['link']}")

print(f"内容已保存至 {output_file}")

正在搜索关键词：齐民要术
[{'link': 'http://www.baidu.com/link?url=gnlL07nSifoTtwirQrlUqI4Z9hp0EkzmSM6FBdlFgjRlue8ggQkpa_vs7MzRJRWCS8tumjz3RwhF4D3bPfivTWuxncucV958J0a58E7F85VONou6M0554y_cXyefWqxdM7aMxHcaVVS9LGCJrOdkLAY9JqEeTogGwf3p_Gphg32ZXp6dw6BdmO3HlyiH1XTA-36P81qVGDpQQrFmgV8tI_',
  'title': '《齐民要术》词汇语法研究 - 百度百科'},
 {'link': 'http://www.baidu.com/link?url=3iRM473zWvElPV5T9RTys2Qwi0y2jnAcDOdl1m_mfmbjtUTfnMsaIRCSa_iZHi206r-ZPZABImE5Z-Z0qLjU-s18BGxVU8aXRTeL1n1ZdmRDdO5bSk6yNAznTTxJxZxP',
  'title': '四大农书 - 百度百科'},
 {'link': 'http://www.baidu.com/link?url=Jjj5OawQy19Z-IL7X8VyTzg6Gd0RTp-AO-QLltVMnmwOHOCA0Qg-dEfRDKRsgOW0YDSXo-WxsfT8bhHTj7Aq2fDWItywOT1Htl6U1gly3Q3ms3dG07cxmn0MSxGPuhiEstt39XRHMKGMF8T_J6kRiOSl09CYeNBhefPoO0NN0m7vQL42m_xp35UJIorBXuT-rKt5vo-n75J1Gn2n1NEEUJ4LnrtCx29HTFAjuD72Qgq',
  'title': '齐民要术讲的是什么内容 - 百度文库'},
 {'link': 'http://www.baidu.com/link?url=Jjj5OawQy19Z-IL7X8VyTzg6Gd0RTp-AO-QLltVMnmwOHOCA0Qg-dEfRDKRsgOW0T0ChchUp48y8L-xInOYn37UeS8Cdni8k7eoc_Jr2nc9bNagXim9oi4qvu0OvTWrGBRWMlqhlnEya4

In [23]:
def clean_data(input_file, output_file, stop_words):
    """清理重复和无意义数据"""
    try:
        with open(input_file, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # 去掉重复行和首尾空白
        unique_lines = list(set(line.strip() for line in lines if line.strip()))

        # 删除无意义的行
        meaningful_lines = [
            line for line in unique_lines
            if not any(word in line for word in stop_words)
        ]

        # 保存清理后的数据
        with open(output_file, "w", encoding="utf-8") as file:
            file.write("\n".join(meaningful_lines))

        print(f"清洗完成！清理后数据保存到：{output_file}")

    except Exception as e:
        print(f"清洗过程中发生错误：{e}")

# 无意义词汇表
stop_words = ["新客立减", "登录", "新建", "上传", "最近", "收藏", "下载"]

# 输入和输出文件路径
input_file = "Book/齐民要术_补充材料.txt"   # 原始数据文件
output_file = "Book/齐民要术_补充材料.txt"  # 清理后数据文件

# 执行清理
clean_data(input_file, output_file, stop_words)
clean_and_combine_txt(input_file, output_file)

清洗完成！清理后数据保存到：Book/齐民要术_补充材料.txt
成功处理文本并保存至：Book/齐民要术_补充材料.txt


In [24]:
with open("Book/齐民要术.txt","r", encoding="utf-8") as file:
    content = file.read()
with open("Book/齐民要术_补充材料.txt","r", encoding="utf-8") as file:
    content += file.read()    
chunker = TextChunker(content)
chunks = chunker.chunk_text(chunk_size=200, overlap=100)
chunks

['前言 引子 本书由“行行”整理，如果你不知道读什么书 或者想获得更多免费电子书请加小编微信或 ：小编也和结交一些喜欢读书的 朋友或者关注小编个人微信公众号名称：幸福的 味道为了方便书友朋友找书和看书，小编自己做 了一个电子书下载网站，网站的名称为：周读网 址：.. 我们的祖国自古“以农立国”，经过悠久历史 岁月的累积，集中了各族人民的智慧，形成了一 个先进的',
 '朋友找书和看书，小编自己做 了一个电子书下载网站，网站的名称为：周读网 址：.. 我们的祖国自古“以农立国”，经过悠久历史 岁月的累积，集中了各族人民的智慧，形成了一 个先进的丰富的完备的农业生产技术科学知 识体系。它是我国文化的一个重要构成部分，也 是我们祖先为人类创造的宝贵遗产之一。纵观历 史，诸多的世界古代文明，都相继淹没在历史的 波�',
 '丰富的完备的农业生产技术科学知 识体系。它是我国文化的一个重要构成部分，也 是我们祖先为人类创造的宝贵遗产之一。纵观历 史，诸多的世界古代文明，都相继淹没在历史的 波涛中，唯有中华文明虽经起伏跌宕，却始终没 有中断过，作为文明古国稳稳地屹立于世界东 方。这世界文化奇观的出现，其深厚的物质基础 是什么？历史告诉我们，任何一个民族或',
 '�中，唯有中华文明虽经起伏跌宕，却始终没 有中断过，作为文明古国稳稳地屹立于世界东 方。这世界文化奇观的出现，其深厚的物质基础 是什么？历史告诉我们，任何一个民族或群体， 如果农业生产中断了，那么他们的文化及历史传 统也会随之中断。在有历史记载的几千年中，我 国农业经历了大大小小无数次天灾人祸的考验， 始终没有出现过因技术指导失误而引起的重大',
 '群体， 如果农业生产中断了，那么他们的文化及历史传 统也会随之中断。在有历史记载的几千年中，我 国农业经历了大大小小无数次天灾人祸的考验， 始终没有出现过因技术指导失误而引起的重大挫 折和失败。这显示了中华农业精耕细作优良传统 的强大生命力，也正是中华文明持续发展的深厚 根基。 在我国精耕细作的农业传统中，蕴藏着丰富 的先进生产技术知识。历代有�',
 '挫 折和失败。这显示了中华农业精耕细作优良传统 的强大生命力，也正是中华文明持续发展的深厚 根基。 在我国精耕细作的农业传统中，蕴藏着丰富 的先进生产技术知识。历代有识之士将这些知识 用文字记录下来，在两千多年的文明发展

# 加载模型，把文本向量化存储

In [25]:
#嵌入模型：将文本向量化
embedding_model = ChineseBERTEmbedding("google-bert/bert-base-chinese")
#向量库
database = Vectordatabase(chunks)

#对字块列表进行，批量的embedded编码，传入embedding模型，返回一个向量列表
Vectors = database.get_vector(embedding_model)
#把向量列表存储到json文件中，把子块列表也存储到json文件
database.persist()

100%|██████████| 8703/8703 [04:15<00:00, 34.10it/s]


# 测试

In [23]:
text = "如何选用优质的稻种"
#嵌入模型：将文本向量化
embedding_model = ChineseBERTEmbedding("google-bert/bert-base-chinese")

database = Vectordatabase()

database.load_vector()

result = database.query(text,embedding_model,Reranker_model=None)
result

[('时令，如何将土地治整成适 合种水稻的稻田，水稻的田间管理，水稻的收获 及稻种的储藏，舂稻米的注意事项等都作了详细 地记述。 ..中水稻先浸种到“芽长二分”再播种的记 叙，可能是有关“浸种”的最早文字记载。 水稻，产量高。因为它能靠水层保持比较稳 定的温度，所以对水的要求很严格。水过多过少 都不能生长，含泥过多的浊水也对生长不利。水 稻种植的关键之',
  0.7861777096925737),
 ('《古逸从书》本《玉 烛宝典》所引崔寔《四民月令》，改为“稻”。 别，分栽。 译文 ..“五月，可以分栽稻和蓝，尽夏至 后二十天为止。” 旱稻第十二 题解 旱稻是对水要求较低的品种，耐旱也耐涝。 它适宜在春旱而夏秋易涝的低洼地区种植， 即“旱稻用下田”。本篇介绍了如何整治种旱稻的 田土，旱稻播种的时令方法以及它',
  0.736043999671903),
 ('要求较低的品种，耐旱也耐涝。 它适宜在春旱而夏秋易涝的低洼地区种植， 即“旱稻用下田”。本篇介绍了如何整治种旱稻的 田土，旱稻播种的时令方法以及它的田间管 理。最后还介绍了高地种旱稻的注意事项。 ..旱稻用下田，白土胜黑土。非言下 田胜高原，但夏停水者，不得禾豆麦。稻， 田种，虽涝亦收；所谓彼此俱获',
  0.7255325860356331),
 ('详细的介绍。 ..介绍可以在葱地里套种一些胡荽。 《齐民要术》中有不少“套作”的例子。“套作”在 我国已有两千年的历史。初期的“套种”，就是在 可用的地面上，同一个时期要保有最大数量的植 株，比较合理的办法是种上两种作物：一种作 物，出苗后不久就可以收获利用，另一种作物一 直在地里生长到成熟。这样，苗期地面可以得到 充分利用；而第二种作物在后期生长时又可以有 充�',
  0.724818641957398),
 ('从准备砧木，选择插条（接穗）起，到正式 接插（嫁接）：包括接时应如何对准形成层“木 边向木，皮还近皮”，使包含形成层在内的“青 皮”相互结合；插接后，如何封闭接口，裹扎和 看护管理，维护愈合后的新个体健康成长，都有 一系列详细正确的技术记载和原理说明。.. 记述用枣或石榴作砧木嫁接，结出的梨品质上 等。梨和枣石榴的',
  0.7160648524136933),
 ('�种培育经验的 总结。 从....讲述了关于树的栽种和培育 的各种注

In [ ]:
#重排序模型
reranker_model = BGEReranker("BAAI/bge-reranker-base")
result = database.query(text,embedding_model,Reranker_model=reranker_model)
result

[('时令，如何将土地治整成适 合种水稻的稻田，水稻的田间管理，水稻的收获 及稻种的储藏，舂稻米的注意事项等都作了详细 地记述。 ..中水稻先浸种到“芽长二分”再播种的记 叙，可能是有关“浸种”的最早文字记载。 水稻，产量高。因为它能靠水层保持比较稳 定的温度，所以对水的要求很严格。水过多过少 都不能生长，含泥过多的浊水也对生长不利。水 稻种植的关键之',
  -0.7782473564147949),
 ('《古逸从书》本《玉 烛宝典》所引崔寔《四民月令》，改为“稻”。 别，分栽。 译文 ..“五月，可以分栽稻和蓝，尽夏至 后二十天为止。” 旱稻第十二 题解 旱稻是对水要求较低的品种，耐旱也耐涝。 它适宜在春旱而夏秋易涝的低洼地区种植， 即“旱稻用下田”。本篇介绍了如何整治种旱稻的 田土，旱稻播种的时令方法以及它',
  -0.8144160509109497),
 ('要求较低的品种，耐旱也耐涝。 它适宜在春旱而夏秋易涝的低洼地区种植， 即“旱稻用下田”。本篇介绍了如何整治种旱稻的 田土，旱稻播种的时令方法以及它的田间管 理。最后还介绍了高地种旱稻的注意事项。 ..旱稻用下田，白土胜黑土。非言下 田胜高原，但夏停水者，不得禾豆麦。稻， 田种，虽涝亦收；所谓彼此俱获',
  -0.9044293165206909),
 ('详细的介绍。 ..介绍可以在葱地里套种一些胡荽。 《齐民要术》中有不少“套作”的例子。“套作”在 我国已有两千年的历史。初期的“套种”，就是在 可用的地面上，同一个时期要保有最大数量的植 株，比较合理的办法是种上两种作物：一种作 物，出苗后不久就可以收获利用，另一种作物一 直在地里生长到成熟。这样，苗期地面可以得到 充分利用；而第二种作物在后期生长时又可以有 充�',
  -1.7714248895645142),
 ('从准备砧木，选择插条（接穗）起，到正式 接插（嫁接）：包括接时应如何对准形成层“木 边向木，皮还近皮”，使包含形成层在内的“青 皮”相互结合；插接后，如何封闭接口，裹扎和 看护管理，维护愈合后的新个体健康成长，都有 一系列详细正确的技术记载和原理说明。.. 记述用枣或石榴作砧木嫁接，结出的梨品质上 等。梨和枣石榴的',
  -0.757671594619751),
 ('�种培育经验的 总结。 从....讲述了关于树的栽种和培

In [ ]:
# LLM智谱大模型API
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)
    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

def ask_glm(key, model, max_tokens, temperature, content):
    # 智谱大模型
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': generate_token(key, 1000)
    }

    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": content
    }

    response = requests.post(url, headers=headers, json=data)

    return response.json()
#你的API
text = text + str(result)
key = "8b00afab1e7440a29b2cc9dba6e17f94.pjG00RsmnmyiHqsG"
content = [{"role": "user", "content": text}]
full_response = ask_glm(key, "glm-3-turbo", 512, 0.8,content)

In [29]:
print(full_response['choices'][0]['message']['content'])

选用优质的稻种是确保水稻产量和质量的关键。以下是一些指导原则和建议，这些原则基于传统农业知识和现代种植实践：

1. **选择适合当地环境的稻种**：不同地区有不同的气候和土壤条件，选择适应当地环境的稻种是非常重要的。应考虑诸如耐寒性、耐旱性、病虫害抵抗力和适应特定生长周期等因素。

2. **浸种和播种**：传统上，水稻种子需要浸种至“芽长二分”后再播种。这有助于提高种子发芽率和植株生长的一致性。现代种植中，使用适当的预处理方法，如激素处理，也可以提高发芽率。

3. **土地整治**：稻田的土壤需要具有良好的排水性和肥力。治整土地使其适合水稻生长，包括深翻土壤、清除杂草和适当的土壤改良，以确保植物有足够的生长空间和必需的营养。

4. **田间管理**：水稻生长过程中需要定期灌溉、施肥、除草和病虫害防治。重要的是要监控稻田的水分状况，避免水过多或过少，以及定期检查植株状况，及时处理问题。

5. **收获和储藏**：收获时间对稻种的品质至关重要。应在水稻成熟且谷粒硬化后及时收获。收获后的稻谷应妥善干燥和储存，以防霉变和发芽。

6. **农业技术文献的参考**：《玉烛宝典》、《齐民要术》等古籍中包含了丰富的农业知识，包括水稻种植的详细指导和注意事项。这些文献对现代农业生产仍具有参考价值。

7. **注意环境可持续性**：在追求高产的同时，应注重环境保护，合理使用化肥和农药，采取综合病虫害管理策略，减少对环境的负面影响。

遵循这些原则和建议，可以提高选择和种植稻种的成功率，从而获得高品质和高产量的水稻。
